In [ ]:
from model.functions import get_stratified_data

train_set, test_set = get_stratified_data()

train_set.head()

In [ ]:
from model.pipelines import get_basic_nn_pipeline

pipeline = get_basic_nn_pipeline()
df = pipeline.fit_transform(train_set)

df.head()

In [ ]:
from sklearn.neighbors import NearestNeighbors


k_value = 3
nn_graph = NearestNeighbors(n_neighbors=k_value, algorithm='brute', metric='jaccard')
nn_graph.fit(df)

distances, indices = nn_graph.kneighbors(df)

/Users/mav/code/personal/canihaveatvshowplz/.venv/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2182: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [ ]:
import networkx as nx

graph = nx.Graph()

for _, row in df.iterrows():
    if row['is_liked'] == 1:
        graph.add_edge(row['user_id'], row['show_id'])

In [ ]:
def generate_recommendations_from_graph(graph, given_show_ids):
    users_liking_given_shows = set()

    # Find users who liked the given shows
    for show_id in given_show_ids:
        if show_id in graph:
            users_liking_given_shows.update(list(graph.neighbors(show_id)))

    recommended_shows = set()

    # Traverse the graph to find other shows liked by these users
    for user in users_liking_given_shows:
        # Assume the graph has connections between users and their liked shows
        user_liked_shows = set(graph.neighbors(user))
        recommended_shows.update(user_liked_shows)

    # Remove the originally provided show_ids from recommendations
    recommended_shows -= set(given_show_ids)

    return list(recommended_shows)

In [ ]:
show_ids = [
    "9c7a19bc-6f54-4d21-b6b6-0606e29fbba3",
    "8c0ebcc3-fbf7-4a67-8c3e-85f13de11f8e",
]

recommended_shows = generate_recommendations_from_graph(graph, show_ids)

print(recommended_shows)

TODO: only output the graph in our model, no longer need ratings.csv...?
```
import pickle

# Assuming 'graph' is your NetworkX graph
with open('your_graph.pkl', 'wb') as f:
    pickle.dump(graph, f)
```